# Sentiment-Analyse von Genios.de Artikel-Metadaten

Dieses Notebook demonstriert, wie man mithilfe des vortrainierten multilingualen Sentiment-Analyse-Modells `tabularisai/multilingual-sentiment-analysis` von Hugging Face die Kombination aus Titel und Kontext-Snippets von Artikeln analysiert, die zuvor von Genios.de gescraped und in einer CSV-Datei gespeichert wurden.

**Wichtig:** Dieses Notebook dient zu Demonstrationszwecken. Im Rahmen des Kurses werden auch die Limitationen dieser Technik erläutert.

## 1. Vorbereitung und CSV-Datei Upload

Stellen Sie sicher, dass Sie die zuvor generierte CSV-Datei (z.B. `genios_artikel_afd_YYYYMMDD_HHMMSS.csv`) in das Arbeitsverzeichnis dieses Colab-Notebooks hochgeladen haben oder den Pfad korrekt angeben.

In [ ]:
# Installiere die 'transformers' Bibliothek, falls noch nicht geschehen
!pip install transformers[sentencepiece] pandas openpyxl

In [ ]:
# Importiere benötigte Bibliotheken
import pandas as pd
from transformers import pipeline
import time

print('Bibliotheken importiert.')

In [ ]:
# Laden der CSV-Daten
# BITTE DEN DATEINAMEN ANPASSEN, falls Ihre Datei anders heißt!
csv_filename = 'genios_afd_articles_20240901_120000.csv' # Beispielname, anpassen!

try:
    df = pd.read_csv(csv_filename)
    print(f'CSV-Datei "{csv_filename}" erfolgreich geladen.')
    print(f'Anzahl der Artikel-Einträge: {len(df)}')
    print(f'Spalten in der CSV: {df.columns.tolist()}')
    # Zeige die ersten paar Zeilen zur Überprüfung
    print("\nErste paar Zeilen der Daten:")
    print(df.head())
except FileNotFoundError:
    print(f"FEHLER: Die Datei '{csv_filename}' wurde nicht gefunden. Bitte laden Sie sie hoch oder passen Sie den Pfad an.")
except Exception as e:
    print(f"Ein Fehler ist beim Laden der CSV-Datei aufgetreten: {e}")

## 2. Deduplizierung (Wichtig!)

Wie im Scraping-Notebook erwähnt, können durch die Fortsetzungslogik Duplikate entstehen. Wir sollten diese vor der Analyse entfernen, um die Ergebnisse nicht zu verfälschen und Rechenzeit zu sparen.

In [ ]:
if 'df' in locals(): # Prüfen, ob DataFrame geladen wurde
    print(f"Datensätze vor Deduplizierung: {len(df)}")
    # Deduplizierung basierend auf einer eindeutigen Artikel-ID und dem Abfragedatum
    # 'document_id_attr' ist wahrscheinlich ein guter Schlüssel.
    # 'query_date' ist wichtig, falls derselbe Artikel an verschiedenen Tagen mit unterschiedlichen Kontexten auftaucht (unwahrscheinlich bei kurzen Zeiträumen).
    if 'document_id_attr' in df.columns and 'query_date' in df.columns:
        df.drop_duplicates(subset=['document_id_attr', 'query_date'], keep='first', inplace=True)
        print(f"Datensätze nach Deduplizierung: {len(df)}")
    else:
        print("Warnung: Spalten 'document_id_attr' oder 'query_date' nicht für Deduplizierung gefunden. Überspringe Deduplizierung.")
else:
    print("DataFrame 'df' nicht geladen. Deduplizierung kann nicht durchgeführt werden.")

## 3. Initialisierung der Sentiment-Analyse Pipeline

In [ ]:
# Initialisiere die Sentiment-Analyse Pipeline
# 'tabularisai/multilingual-sentiment-analysis' ist ein gutes mehrsprachiges Modell
print('Initialisiere Sentiment-Analyse Pipeline... Dies kann einen Moment dauern, da das Modell heruntergeladen wird.')
try:
    sentiment_pipe = pipeline(
        'text-classification', 
        model='tabularisai/multilingual-sentiment-analysis', 
        # device=0 # Uncomment für GPU-Nutzung, falls verfügbar und CUDA eingerichtet ist
    )
    print('Sentiment-Analyse Pipeline erfolgreich initialisiert.')
except Exception as e:
    print(f"Fehler bei der Initialisierung der Pipeline: {e}")
    print("Stellen Sie sicher, dass Sie eine Internetverbindung haben und die Transformers-Bibliothek korrekt installiert ist.")

## 4. Analysefunktion und Durchführung

In [ ]:
def analyze_sentiment_for_text(text_to_analyze):
    """Analysiert den Sentiment eines Textes und gibt das Ergebnis zurück."""
    if not text_to_analyze or pd.isna(text_to_analyze): # Prüft auf leere Strings oder NaN
        return {'label': 'NO_TEXT', 'score': 0.0}
    try:
        # Das Modell kann lange Texte verarbeiten, aber sehr lange Texte können zu 
        # Out-of-Memory-Fehlern führen oder die maximale Sequenzlänge überschreiten.
        # Wir kürzen den Text ggf. auf eine vernünftige Länge (z.B. die ersten 512 Tokens des Modells).
        # Die Pipeline sollte das intern handhaben, aber zur Sicherheit:
        max_length = 500 # Zeichen, nicht Tokens, für eine einfache Kürzung
        truncated_text = text_to_analyze[:max_length]
        
        result = sentiment_pipe(truncated_text)
        if result and isinstance(result, list):
            return result[0] # Das Ergebnis ist eine Liste mit einem Dictionary
        else:
            return {'label': 'Error_Parsing_Result', 'score': 0.0}
    except Exception as e:
        print(f"Fehler bei der Analyse von Text: '{str(text_to_analyze)[:50]}...': {e}")
        return {'label': 'Error_Analysis', 'score': 0.0}

In [ ]:
if 'df' in locals() and 'sentiment_pipe' in locals():
    print("Führe Sentiment-Analyse für Titel und Kontext-Snippets durch...")
    
    # Listen für die Ergebnisse
    sentiment_labels = []
    sentiment_scores = []
    
    start_time = time.time()
    
    # Iteriere über die Zeilen des DataFrames
    # Begrenze für Demo-Zwecke die Anzahl der zu analysierenden Artikel, um Zeit zu sparen
    # Entferne oder erhöhe .head(X), um alle Artikel zu analysieren
    # for index, row in df.head(20).iterrows(): 
    for index, row in df.iterrows(): 
        title = str(row.get('title', ''))
        context = str(row.get('context_snippet', ''))
        
        # Kombiniere Titel und Kontext für eine umfassendere Analyse
        # Füge ein Trennzeichen hinzu, falls beide vorhanden sind
        combined_text = title
        if title and context:
            combined_text += " [SEP] " + context
        elif context: # Nur Kontext, kein Titel
            combined_text = context
        
        if not combined_text.strip(): # Wenn nach Kombination immer noch leer
            print(f"Artikel {index+1} (ID: {row.get('document_id_attr', 'N/A')}): Kein Text für Analyse vorhanden.")
            sentiment_result = {'label': 'NO_TEXT', 'score': 0.0}
        else:
            sentiment_result = analyze_sentiment_for_text(combined_text)
            print(f"Artikel {index+1} (ID: {row.get('document_id_attr', 'N/A')}) - Sentiment: {sentiment_result['label']} (Score: {sentiment_result.get('score', 0.0):.4f})")
        
        sentiment_labels.append(sentiment_result['label'])
        sentiment_scores.append(sentiment_result.get('score', 0.0))
        
        # Statusausgabe alle X Artikel
        if (index + 1) % 50 == 0:
            elapsed_time = time.time() - start_time
            print(f"--- {index+1} Artikel verarbeitet in {elapsed_time:.2f} Sekunden ---")

    # Füge die Ergebnisse als neue Spalten zum DataFrame hinzu
    df['sentiment_label'] = sentiment_labels
    df['sentiment_score'] = sentiment_scores
    
    end_time = time.time()
    print(f'\nSentiment-Analyse für {len(df)} Einträge abgeschlossen in {(end_time - start_time):.2f} Sekunden.')
    
    # Zeige die ersten Zeilen mit den neuen Sentiment-Spalten
    print("\nDataFrame mit Sentiment-Ergebnissen:")
    print(df[['title', 'context_snippet', 'sentiment_label', 'sentiment_score']].head())
    
    # Speichere den DataFrame mit den Sentiment-Ergebnissen (optional)
    output_filename = f"genios_artikel_mit_sentiment_{csv_filename.split('_')[-2]}_{csv_filename.split('_')[-1]}"
    df.to_csv(output_filename, index=False, encoding='utf-8')
    # Alternativ als Excel, falls bevorzugt:
    # df.to_excel(output_filename.replace('.csv', '.xlsx'), index=False)
    print(f"\nDaten mit Sentiment-Analyse gespeichert in: {output_filename}")

else:
    print("DataFrame 'df' oder 'sentiment_pipe' nicht initialisiert. Analyse kann nicht durchgeführt werden.")

## 5. Beispielhafte Auswertung

Nachdem die Sentiment-Analyse durchgeführt wurde, können wir uns die Verteilung der Sentiments ansehen.

In [ ]:
if 'df' in locals() and 'sentiment_label' in df.columns:
    print("\nVerteilung der Sentiment-Labels:")
    sentiment_counts = df['sentiment_label'].value_counts()
    print(sentiment_counts)
    
    # Einfache Visualisierung (optional, erfordert matplotlib)
    try:
        import matplotlib.pyplot as plt
        sentiment_counts.plot(kind='bar', title='Verteilung der Sentiment-Labels')
        plt.ylabel('Anzahl Artikel')
        plt.xlabel('Sentiment Label')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    except ImportError:
        print("Matplotlib nicht installiert. Für eine Visualisierung bitte installieren: !pip install matplotlib")
else:
    print("DataFrame oder Sentiment-Spalten nicht vorhanden für die Auswertung.")

## Fazit

In diesem Notebook haben wir gezeigt, wie man:

- Eine CSV-Datei mit gescrapten Artikel-Metadaten einliest und dedupliziert.
- Die Kombination aus Artikeltitel und Kontext-Snippet für die Sentiment-Analyse vorbereitet.
- Mithilfe eines vortrainierten, multilingualen Sentiment-Analysemodells von Hugging Face die Stimmung der kombinierten Texte klassifiziert.
- Die Ergebnisse zurück in den DataFrame schreibt und optional speichert.
- Eine einfache Auswertung der Sentiment-Verteilung durchführt.

**Hinweis:** Dies ist ein Demonstrationscode. In einem echten Einsatzszenario sind weitere Vorverarbeitungsschritte, Fehlerbehandlungen und tiefgreifendere Analysen (z.B. Betrachtung der Scores, zeitliche Entwicklung etc.) ratsam. Die Qualität der Sentiment-Analyse hängt stark vom Modell und der Qualität/Länge der Eingabetexte ab.